# 第五章 例5.5.1  RAS 平衡法

In [179]:
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Alias, Domain,Sense,Smax
from gamspy.math import abs, Max
import pandas as pd
pd.options.display.float_format = '{:.1f}'.format

### Container

In [180]:
m = Container()

### Sets

In [181]:
ac = Set(container=m, name="ac", description="all units", records=["sec1","sec2","lab","hh","total","tartot"])
acc = Alias(m, name="acc", alias_with=ac)
i = Set(container=m, name="i", domain=ac, description="units", records=["sec1","sec2","lab","hh"])
j = Alias(m, name="j", alias_with=i)
# total = Set(container=m, name="total", domain=ac, description="total", records="total")
# tartot = Set(container=m, name="tartot", domain=ac, description="target total", records="tartot")

### Data

In [182]:
data = pd.read_excel("ch5-5-1.xlsx", index_col=0)
data = data.fillna(0)
data = data.stack().reset_index()


### Parameters

In [183]:
sam = Parameter(
    container=m,
    name="sam",
    domain=[ac,acc],
    description="intial value",
    records=data
)
sam.records


,level_0,level_1,value
0,sec1,sec1,52.0
1,sec1,sec2,45.0
2,sec1,lab,0.0
3,sec1,hh,150.0
4,sec1,total,247.0
5,sec1,tartot,270.0
6,sec2,sec1,95.0
7,sec2,sec2,48.0
8,sec2,lab,0.0
9,sec2,hh,90.0


In [184]:
rowdis = Parameter(
    container=m,
    name="rowdis",
    domain=i,
    description="row difference"
)

condis = Parameter(
    container=m,
    name="condis",
    domain=j,
    description="column difference"
)

maxdis = Parameter(
    container=m,
    name="maxdis",
    description="maximum difference"
)


### 循环

In [185]:
maxdis[...] = 0.1
iter = 1
while iter<5000 and maxdis.records.values > 1e-10:
    # column 
    sam["total",j] = Sum(i,sam[i,j])
    sam[i,j] = sam[i,j]/sam["total",j]*sam["tartot",j]
    # row
    sam[i,"total"] = Sum(j,sam[i,j])
    sam[i,j] = sam[i,j]/sam[i,"total"]*sam[i,"tartot"]
    # 检验本循环调整后的最大误差
    condis[j]=abs(Sum(i,sam[i,j])-sam['tartot',j])
    rowdis[i]=abs(Sum(j,sam[i,j])-sam[i,'tartot'])
    # 比较这一轮结果中的最大值
    maxdis[...]=Max( Smax(i,rowdis[i]),Smax(j,condis[j]) )

    iter=iter+1

### 结果

In [186]:
sam.records

,ac,acc,value
0,sec1,sec1,62.8
1,sec1,sec2,68.6
2,sec1,hh,138.6
3,sec1,total,270.0
4,sec1,tartot,270.0
5,sec2,sec1,98.7
6,sec2,sec2,62.9
7,sec2,hh,71.4
8,sec2,total,233.0
9,sec2,tartot,233.0


In [187]:
results = sam.records.loc[:,["ac","acc","value"]].pivot(index="ac",columns="acc",values="value")
results

acc,sec1,sec2,lab,hh,total,tartot
ac,,,,,,
sec1,62.8,68.6,NaN,138.6,270.0,270.0
sec2,98.7,62.9,NaN,71.4,233.0,233.0
lab,108.5,101.5,NaN,NaN,210.0,210.0
hh,NaN,NaN,210.0,NaN,210.0,210.0
total,270.0,233.0,210.0,210.0,NaN,NaN
tartot,270.0,233.0,210.0,210.0,NaN,NaN


## 如果使用python，不使用gamspy

In [188]:
import numpy as np
data = pd.read_excel("ch5-5-1.xlsx", index_col=0)
data

,sec1,sec2,lab,hh,total,tartot
sec1,52.0,45.0,NaN,150.0,247.0,270.0
sec2,95.0,48.0,NaN,90.0,233.0,233.0
lab,120.0,89.0,NaN,NaN,209.0,210.0
hh,NaN,NaN,192.0,NaN,192.0,210.0
total,267.0,182.0,192.0,240.0,NaN,NaN
tartot,270.0,233.0,210.0,210.0,NaN,NaN


In [189]:
section =  data.loc["sec1":"hh","sec1":"hh"].values
section

array([[ 52.,  45.,  nan, 150.],
       [ 95.,  48.,  nan,  90.],
       [120.,  89.,  nan,  nan],
       [ nan,  nan, 192.,  nan]])

In [190]:
tartot = data.loc["sec1":"hh","tartot"].values
tartot

array([270., 233., 210., 210.])

### 循环

In [191]:
maxdis = 0.1
iter = 1
while iter<5000 and maxdis> 1e-10:
    # column 
    section = section / np.nansum(section,0) * tartot
    # row
    section = (section / np.nansum(section,1)[:,None]) * tartot[:,None]
    # 检验本循环调整后的最大误差
    condis=np.absolute((np.nansum(section,0)-tartot)).max()
    rowdis=np.absolute(np.nansum(section,1)-tartot[:,None]).max()
    # 比较这一轮结果中的最大值
    maxdis=max(condis,rowdis)
    iter=iter+1

In [192]:
section

array([[ 62.84161483,  68.60350554,          nan, 138.55487963],
       [ 98.66590293,  62.8889767 ,          nan,  71.44512037],
       [108.49248224, 101.50751776,          nan,          nan],
       [         nan,          nan, 210.        ,          nan]])

In [193]:
np.nansum(section,0)

array([270., 233., 210., 210.])

In [194]:
np.nansum(section,1)

array([270., 233., 210., 210.])